In [1]:
using NBInclude
nbinclude("../src/PoissonProcessEstimation.ipynb")
import PoissonProcessEstimation

In [2]:
# Set up kernels
KernList = [MLKernels.GaussianKernel(400.0), MLKernels.GaussianKernel(200.0), MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(50.0)]

4-element Array{MLKernels.ExponentialKernel{Float64,:γ1},1}:
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=400.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=200.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=100.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=50.0,γ=1.0) 

In [22]:
# Initialise params
θ_true = PoissonProcessEstimation.create_rand_params(10,30,length(KernList),[30,15,10,8]);
# Set the log firing rate
θ_true.b = log(100)*ones(size(θ_true.b)); 
θ_true.C = randn(size(θ_true.C))*1;
C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_true);

In [23]:
# Simulate from the model
data = PoissonProcessEstimation.simulate(θ_true, KernList);

Simulating trial 1
Simulating trial 2
Simulating trial 3
Simulating trial 4
Simulating trial 5
Simulating trial 6
Simulating trial 7
Simulating trial 8
Simulating trial 9
Simulating trial 10


In [34]:
# Evaluate the log_likelihood under true parameters
ll = PoissonProcessEstimation.log_likelihood(data, θ_true, KernList)[1]/(N*M)

1293.9540562551915

In [29]:
θ = PoissonProcessEstimation.create_rand_params(size(θ_true.α,1),size(θ_true.C,1),length(KernList),[30,15,10,8]);
θ.C = randn(size(θ.C))*0.5
θ.b = deepcopy(θ_true.b);

ll = PoissonProcessEstimation.log_likelihood(data, θ, KernList)[1]/(N*M)

287007.81497434847

In [30]:
θ_orig = θ
θ_opt = deepcopy(θ)
C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_opt);

In [42]:
@time Ktu, dtKtu, ddtKtu, Kuu, I = 
    PoissonProcessEstimation.optimise!(data, θ_opt, KernList, η_RKHS=1e1, η_Cm=1e1, num_iters = 25, verbose=false);

  9.308181 seconds (256.01 M allocations: 3.992 GB, 5.50% gc time)
Kernels have been built

Starting optimisation, initial log-likelihood is 16619.75442867643

α and C are set, log-likelihood is 142480.87233324884

Optimisation done, final log-likelihood is 361462.51382262295

206.037862 seconds (3.12 G allocations: 60.703 GB, 5.97% gc time)


In [45]:
ll = PoissonProcessEstimation.log_likelihood(data, θ_opt, KernList)[1]/(N*M)

1204.8750460754097

In [44]:
1204.2896*N*M

361286.88

In [46]:
[θ_opt.b θ_true.b]

30x2 Array{Float64,2}:
 4.67488  4.60517
 3.29291  4.60517
 4.42971  4.60517
 4.1119   4.60517
 4.42972  4.60517
 4.17875  4.60517
 3.96626  4.60517
 3.94726  4.60517
 4.63684  4.60517
 3.38952  4.60517
 4.42887  4.60517
 3.08731  4.60517
 4.25188  4.60517
 ⋮               
 4.49756  4.60517
 4.68293  4.60517
 4.31469  4.60517
 3.69903  4.60517
 4.22555  4.60517
 4.71341  4.60517
 3.40157  4.60517
 4.73271  4.60517
 4.61346  4.60517
 4.43585  4.60517
 4.57074  4.60517
 4.4665   4.60517

In [50]:
PoissonProcessEstimation.plot_params(data, θ_opt, θ_true, KernList, num_neur=1, trials=1:10, dim_latent=[])

In [51]:
# Check for subspace angle optimisation
[PoissonProcessEstimation.subspace(θ_opt.C, θ_true.C)[1] PoissonProcessEstimation.subspace(θ_orig.C, θ_true.C)[1]]

4x2 Array{Float64,2}:
  4.05687  62.2354
  7.00546  67.2385
 10.0668   75.6441
 11.783    87.2041

In [52]:
θ_opt.b = zeros(size(θ_opt.b))

# Check for correlation of predicted firing rates
opt_corrs = zeros(N,M)
rand_corrs = zeros(N,M)
rate_true = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_true, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
rate_est = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_opt, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
rate_rand = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_orig, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
for n = 1:N
    for m = 1:M
        opt_corrs[n,m] = cor(rate_true[n,m], rate_est[n,m])
        rand_corrs[n,m] = cor(rate_true[n,m], rate_rand[n,m])
    end
end

# Show the computed average correlation per neuron
[mean(opt_corrs,1)' mean(rand_corrs,1)']

30x2 Array{Float64,2}:
 0.917558  -0.0429862
 0.94106    0.0577879
 0.926246   0.0925703
 0.946805  -0.242842 
 0.914129  -0.0895106
 0.912076  -0.10791  
 0.934735  -0.0466818
 0.936566  -0.138076 
 0.908162   0.0799919
 0.940037   0.0653441
 0.899443  -0.0109574
 0.941785  -0.0940769
 0.919429  -0.189881 
 ⋮                   
 0.911126   0.109266 
 0.917087   0.191636 
 0.898826  -0.110759 
 0.904921  -0.0111467
 0.926058  -0.0758313
 0.920599   0.0520202
 0.939762   0.022063 
 0.897975  -0.0127079
 0.890313  -0.0490552
 0.920106  -0.0983807
 0.901289   0.141718 
 0.908659   0.0599108